In [16]:
import os
import json
import subprocess
import tempfile
import shutil

In [17]:
def get_dvc_diff(old_commit, new_commit=None):
    """
    Получение изменений DVC между двумя коммитами.

    :param old_commit: Старый Git-коммит (a_rev).
    :param new_commit: Новый Git-коммит (b_rev). Если None, сравнивается с текущим workspace.
    :return: Словарь с изменениями.
    """
    # Формируем базовую команду
    cmd = ["dvc", "diff", "--json"]

    # Добавляем старый и новый коммиты, если они указаны
    if old_commit:
        cmd.append(old_commit)
    if new_commit:
        cmd.append(new_commit)

    # Выполняем команду
    result = subprocess.run(cmd, capture_output=True, text=True)

    # Проверяем наличие ошибок
    if result.returncode != 0:
        raise RuntimeError(f"Error running 'dvc diff': {result.stderr.strip()}")

    # Возвращаем результат в виде словаря
    return json.loads(result.stdout)

In [18]:
def calculate_code_changes(old_commit, new_commit):
    """
    Подсчёт изменений в строках кода (.py файлы) между двумя коммитами.

    :param old_commit: Старый Git-коммит.
    :param new_commit: Новый Git-коммит.
    :return: Словарь с количеством добавленных и удалённых строк.
    """
    cmd = ["git", "diff", "--numstat", f"{old_commit}..{new_commit}", "--", "*.py"]
    result = subprocess.run(cmd, capture_output=True, text=True)

    if result.returncode != 0:
        raise RuntimeError(f"Error running 'git diff': {result.stderr.strip()}")

    added_lines = 0
    deleted_lines = 0

    # Разбираем строки вывода
    for line in result.stdout.strip().split("\n"):
        if not line.strip():
            continue
        parts = line.split("\t")
        added_lines += int(parts[0]) if parts[0].isdigit() else 0
        deleted_lines += int(parts[1]) if parts[1].isdigit() else 0

    return {"added": added_lines, "deleted": deleted_lines}


def calculate_data_changes(dvc_diff):
    """
    Подсчёт изменений в данных на основе DVC diff.

    :param dvc_diff: Результат функции get_dvc_diff.
    :return: Словарь с добавленными, удалёнными и модифицированными данными.
    """
    changes = {"added": 0, "deleted": 0, "modified": 0}

    # Обрабатываем добавленные файлы
    for item in dvc_diff.get("added", []):
        path = item["path"]
        if os.path.isfile(path):
            changes["added"] += os.path.getsize(path)

    # Обрабатываем удалённые файлы
    for item in dvc_diff.get("deleted", []):
        # Пример: вместо размера, можно учитывать другое свойство
        changes["deleted"] += 1

    # Обрабатываем модифицированные файлы
    for item in dvc_diff.get("modified", []):
        old_hash = item.get("hash", {}).get("old")
        new_hash = item.get("hash", {}).get("new")
        if old_hash and new_hash:
            # Здесь можно добавить более сложный анализ
            changes["modified"] += 1

    return changes


def analyze_changes(old_commit, new_commit):
    """
    Анализ изменений в коде и данных между двумя коммитами.

    :param old_commit: Старый Git-коммит.
    :param new_commit: Новый Git-коммит.
    :return: Сводная информация об изменениях.
    """
    # Получаем изменения кода
    code_changes = calculate_code_changes(old_commit, new_commit)

    # Получаем изменения данных
    dvc_diff = get_dvc_diff(old_commit, new_commit)
    data_changes = calculate_data_changes(dvc_diff)

    # Объединяем результаты
    return {"code_changes": code_changes, "data_changes": data_changes}

In [25]:
def save_metrics_to_file(commit, metrics, output_dir):
    """
    Сохраняет метрики для заданного коммита в JSON файл.

    :param commit: Хэш коммита.
    :param metrics: Метрики (словарь).
    :param output_dir: Папка для сохранения.
    """
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{commit}.json")
    with open(output_path, "w") as f:
        json.dump(metrics, f, indent=4)


def get_git_commits(repo_path, n):
    """
    Получает последние n коммитов в репозитории.

    :param repo_path: Путь к репозиторию.
    :param n: Количество последних коммитов.
    :return: Список хэшей коммитов (от старых к новым).
    """
    cmd = ["git", "log", "--format=%H", f"-n{n}"]
    result = subprocess.run(cmd, cwd=repo_path, capture_output=True, text=True)

    if result.returncode != 0:
        raise RuntimeError(f"Error running 'git log': {result.stderr.strip()}")

    return result.stdout.strip().split("\n")


def count_lines_in_file(file_path):
    """
    Подсчитывает количество строк в файле.

    :param file_path: Путь к файлу.
    :return: Количество строк.
    """
    try:
        with open(file_path, "r") as f:
            return sum(1 for _ in f)
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return 0


def analyze_changes(repo_path, old_commit, new_commit):
    """
    Анализирует изменения между двумя коммитами.

    :param repo_path: Путь к репозиторию.
    :param old_commit: Старый коммит.
    :param new_commit: Новый коммит.
    :return: Метрики изменений.
    """
    # Переключаемся на старый коммит
    subprocess.run(["git", "checkout", old_commit], cwd=repo_path, check=True)
    subprocess.run(["dvc", "checkout"], cwd=repo_path, check=True)

    # Сохраняем состояние файлов в старом коммите
    old_state = {}
    for root, _, files in os.walk(os.path.join(repo_path, "data")):
        for file in files:
            file_path = os.path.join(root, file)
            old_state[file_path] = count_lines_in_file(file_path)

    # Переключаемся на новый коммит
    subprocess.run(["git", "checkout", new_commit], cwd=repo_path, check=True)
    subprocess.run(["dvc", "checkout"], cwd=repo_path, check=True)

    # Сохраняем состояние файлов в новом коммите
    new_state = {}
    for root, _, files in os.walk(os.path.join(repo_path, "data")):
        for file in files:
            file_path = os.path.join(root, file)
            new_state[file_path] = count_lines_in_file(file_path)

    # Анализируем изменения
    added = 0
    deleted = 0
    modified = 0

    all_files = set(old_state.keys()).union(set(new_state.keys()))
    for file_path in all_files:
        old_lines = old_state.get(file_path, 0)
        new_lines = new_state.get(file_path, 0)

        if old_lines == 0 and new_lines > 0:
            added += new_lines
        elif old_lines > 0 and new_lines == 0:
            deleted += old_lines
        elif old_lines != new_lines:
            modified += abs(new_lines - old_lines)

    return {
        "added_lines": added,
        "deleted_lines": deleted,
        "modified_lines": modified,
    }


def analyze_commit_series(n, output_dir):
    """
    Анализирует последние n коммитов и записывает результаты в папку output_dir.

    :param n: Количество последних коммитов.
    :param output_dir: Папка для сохранения результатов.
    """
    commits = get_git_commits(n)

    with tempfile.TemporaryDirectory() as temp_repo:
        # Клонируем текущий репозиторий во временную директорию
        subprocess.run(["git", "clone", ".", temp_repo], check=True)

        for i in range(len(commits) - 1):
            old_commit = commits[i]
            new_commit = commits[i + 1]

            print(f"Analyzing commit: {new_commit} (compare with {old_commit})")

            try:
                # Анализируем изменения между коммитами
                metrics = analyze_changes(temp_repo, old_commit, new_commit)

                # Сохраняем результаты в файл
                save_metrics_to_file(new_commit, metrics, output_dir)

            except Exception as e:
                print(f"Error analyzing commit {new_commit}: {e}")

In [20]:
analyze_commit_series(2, "output")

Клонирование в «/var/folders/72/p2x4tpws2gl6wkm3bp268fr00000gn/T/tmpdmc88t5r»...
готово.
Примечание: переключение на «a3118823752fd0d2a35586635c2a07ba3d90e624».

Вы сейчас в состоянии «отсоединённого указателя HEAD». Можете осмотреться,
внести экспериментальные изменения и зафиксировать их, также можете
отменить любые коммиты, созданные в этом состоянии, не затрагивая другие
ветки, переключившись обратно на любую ветку.

Если хотите создать новую ветку для сохранения созданных коммитов, можете
сделать это (сейчас или позже), используя команду switch с параметром -c.
Например:

  git switch -c <новая-ветка>

Или отмените эту операцию с помощью:

  git switch -

Отключите этот совет, установив переменную конфигурации
advice.detachedHead в значение false

HEAD сейчас на a311882 notebook


Analyzing commit: 948967da5382375bb6e4073690e33fb83a7390ed (compare with a3118823752fd0d2a35586635c2a07ba3d90e624)


ERROR: Checkout failed for following targets:
data/prepared
data/features
eval
model.pkl
data/data.xml
Is your cache up to date?
<https://error.dvc.org/missing-files>


Error analyzing commit 948967da5382375bb6e4073690e33fb83a7390ed: Command '['dvc', 'checkout']' returned non-zero exit status 255.


In [26]:
def collect_file_states(repo_path):
    """
    Собирает состояние файлов в репозитории.

    :param repo_path: Путь к репозиторию.
    :return: Словарь с состоянием файлов.
    """
    file_states = {}
    for root, _, files in os.walk(os.path.join(repo_path, "data")):
        for file in files:
            file_path = os.path.join(root, file)
            file_states[file_path] = count_lines_in_file(file_path)
    return file_states


def analyze_changes_between_states(old_state, new_state):
    """
    Анализирует изменения между двумя состояниями файлов.

    :param old_state: Состояние файлов в старом коммите.
    :param new_state: Состояние файлов в новом коммите.
    :return: Метрики изменений.
    """
    added = 0
    deleted = 0
    modified = 0

    all_files = set(old_state.keys()).union(set(new_state.keys()))
    for file_path in all_files:
        old_lines = old_state.get(file_path, 0)
        new_lines = new_state.get(file_path, 0)

        if old_lines == 0 and new_lines > 0:
            added += new_lines
        elif old_lines > 0 and new_lines == 0:
            deleted += old_lines
        elif old_lines != new_lines:
            modified += abs(new_lines - old_lines)

    return {
        "added_lines": added,
        "deleted_lines": deleted,
        "modified_lines": modified,
    }

In [ ]:
import tempfile
import shutil
import subprocess
import os
import json


def clone_and_prepare_repo(repo_path, branch=None):
    """Клонирует репозиторий в временную папку."""
    temp_dir = tempfile.mkdtemp()
    subprocess.run(["git", "clone", repo_path, temp_dir], check=True)
    if branch:
        subprocess.run(["git", "checkout", branch], cwd=temp_dir, check=True)
    return temp_dir


def pull_dvc_cache(repo_dir):
    """Пытается загрузить кэш DVC, если возможно."""
    try:
        subprocess.run(["dvc", "pull"], cwd=repo_dir, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Warning: Could not pull DVC cache. Error: {e}")


def analyze_commit_series_with_cache(repo_path, n, output_dir):
    """
    Анализирует последние n коммитов и записывает результаты в output_dir.
    Работает с временным клоном репозитория.
    """
    temp_repo = clone_and_prepare_repo(repo_path)
    try:
        commits = get_git_commits(temp_repo, n)
        os.makedirs(output_dir, exist_ok=True)

        for i in range(len(commits) - 1):
            old_commit = commits[i]
            new_commit = commits[i + 1]
            print(f"Analyzing commit: {new_commit} (compare with {old_commit})")

            try:
                subprocess.run(
                    ["git", "checkout", old_commit], cwd=temp_repo, check=True
                )
                pull_dvc_cache(temp_repo)  # Попытка загрузить кэш для старого коммита
                old_state = collect_file_states(temp_repo)

                subprocess.run(
                    ["git", "checkout", new_commit], cwd=temp_repo, check=True
                )
                pull_dvc_cache(temp_repo)  # Попытка загрузить кэш для нового коммита
                new_state = collect_file_states(temp_repo)

                metrics = analyze_changes_between_states(old_state, new_state)
                save_metrics_to_file(new_commit, metrics, output_dir)

            except Exception as e:
                print(f"Error analyzing commit {new_commit}: {e}")

    finally:
        shutil.rmtree(temp_repo)

In [37]:
def analyze_two_commits_with_cache(repo_path, old_commit, new_commit, output_dir):
    """
    Анализирует изменения между двумя коммитами и записывает результаты в output_dir.
    Работает с временным клоном репозитория.
    """
    temp_repo = clone_and_prepare_repo(repo_path)
    try:
        os.makedirs(output_dir, exist_ok=True)
        print(f"Analyzing commit: {new_commit} (compare with {old_commit})")

        try:
            subprocess.run(["git", "checkout", old_commit], cwd=temp_repo, check=True)
            pull_dvc_cache(temp_repo)  # Попытка загрузить кэш для старого коммита
            old_state = collect_file_states(temp_repo)

            subprocess.run(["git", "checkout", new_commit], cwd=temp_repo, check=True)
            pull_dvc_cache(temp_repo)  # Попытка загрузить кэш для нового коммита
            new_state = collect_file_states(temp_repo)

            metrics = analyze_changes_between_states(old_state, new_state)
            save_metrics_to_file(new_commit, metrics, output_dir)

        except Exception as e:
            print(f"Error analyzing commit {new_commit}: {e}")

    finally:
        shutil.rmtree(temp_repo)

In [43]:
def compare_branches(repo_path, branch1, branch2, output_dir):
    """
    Сравнивает HEAD коммиты двух веток и записывает результаты в output_dir.

    :param repo_path: Путь к репозиторию.
    :param branch1: Первая ветка.
    :param branch2: Вторая ветка.
    :param output_dir: Папка для сохранения результатов.
    """
    temp_repo = clone_and_prepare_repo(repo_path)
    try:
        # Получаем последний коммит для каждой ветки
        subprocess.run(
            ["git", "checkout", branch1],
            cwd=temp_repo,
            check=True,
        )
        head_commit_branch1 = subprocess.run(
            ["git", "rev-parse", "HEAD"], cwd=temp_repo, capture_output=True, text=True
        ).stdout.strip()

        subprocess.run(
            ["git", "checkout", branch2],
            cwd=temp_repo,
            check=True,
        )
        head_commit_branch2 = subprocess.run(
            ["git", "rev-parse", "HEAD"], cwd=temp_repo, capture_output=True, text=True
        ).stdout.strip()

        # Анализируем изменения между коммитами
        analyze_two_commits_with_cache(
            repo_path, head_commit_branch1, head_commit_branch2, output_dir
        )

    except Exception as e:
        print(f"Error comparing branches {branch1} and {branch2}: {e}")

    finally:
        shutil.rmtree(temp_repo)

In [44]:
compare_branches(".", "main", "2-track-data", "output")

Клонирование в «/var/folders/72/p2x4tpws2gl6wkm3bp268fr00000gn/T/tmp5gslx1rz»...
готово.
Уже на «main»
Примечание: переключение на «2-track-data».

Вы сейчас в состоянии «отсоединённого указателя HEAD». Можете осмотреться,
внести экспериментальные изменения и зафиксировать их, также можете
отменить любые коммиты, созданные в этом состоянии, не затрагивая другие
ветки, переключившись обратно на любую ветку.

Если хотите создать новую ветку для сохранения созданных коммитов, можете
сделать это (сейчас или позже), используя команду switch с параметром -c.
Например:

  git switch -c <новая-ветка>

Или отмените эту операцию с помощью:

  git switch -

Отключите этот совет, установив переменную конфигурации
advice.detachedHead в значение false

HEAD сейчас на b1a30e6 Add raw data
Клонирование в «/var/folders/72/p2x4tpws2gl6wkm3bp268fr00000gn/T/tmpg6sb0ap8»...
готово.
Примечание: переключение на «12320f07732430137af47ca0af98991b3dd62a5d».

Вы сейчас в состоянии «отсоединённого указателя HEAD»

Эта ветка соответствует «origin/main».
Analyzing commit: b1a30e6795466131f80994a52c1d4aa145641698 (compare with 12320f07732430137af47ca0af98991b3dd62a5d)
A       model.pkl
A       eval/
A       data/features/
A       data/prepared/
A       data/data.xml
5 files added and 17 files fetched
Error reading file /var/folders/72/p2x4tpws2gl6wkm3bp268fr00000gn/T/tmpg6sb0ap8/data/features/train.pkl: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte
Error reading file /var/folders/72/p2x4tpws2gl6wkm3bp268fr00000gn/T/tmpg6sb0ap8/data/features/test.pkl: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte


Предыдущая позиция HEAD была 12320f0 up
HEAD сейчас на b1a30e6 Add raw data


D       model.pkl
D       eval/
D       data/features/
D       data/prepared/
4 files deleted


No remote provided and no default remote set.
